In [19]:
# 전반적인 story line
# (1) 데이터 읽어오기
# (2) 데이터 증강(불균형 데이터 처리)
	# - oversampling을 해서 균형을 맞추고 -> 증강을 하면 되지 않을까?
# (3) 데이터 분할
# (4) 모델 학습
	# <Deep Learning>: 알고리즘을 계층으로 구성하여 자체적으로 배우고 지능적인 결정을 내림
		# - 범주형, 수치형 특성 분리 후 전처리
		# - [Train] 손실함수(loss) 정의 -> model 정의
		# - [Test] test dataset을 불러와서 -> 예측 결과를 저장할 리스트

	# <Machine Learning>: 학습한 내용에 따라 정보에 근거한 결정을 내림
		# - light GBM, XG boost, cat boost
		# 주로 ML classification에서는 decision tree를 활용: 그 based model임

In [20]:
# 데이터 증강(불균형 데이터 처리): CTGAN, TabNet, SMOTE (undersampling -> oversampling)
	# - sampling: 주어진 확률 분포에 기반해 대응하는 랜덤 사건을 만들어내는 것	
	# - oversampling: 불균형 데이터를 균형을 맟추는 것
	# - 증강: 훈련 데이터가 부족할 때 데이터를 늘려주는 방법
	# - oversampling을 해서 균형을 맞추고 -> 증강을 하면 되지 않을까?

In [21]:
# 데이터 스케일링: 서로 다른 변수의 값 범위를 일정한 수준으로 맞추는 작업
	# (1) Standard Scaler
	# (2) Normalizer
	# (3) MinMax Scaler
	# (4) Robust Scaler

In [22]:
# 범주형 변수의 encoding: 범주형 변수를 그대로 넣으면 모델이 인식하지 못하고 에러를 발생시킴
	# - 범주형 변수를 수치형 변수로 인코딩하는 과정이 필요
	# - encoding 방법
		# (1) label encoding
		# (2) one hot encoding
		# (3) ordinal encoding
		# (4) binary encoding
		# (5) woe encoding
		# (6) James-Stein

In [23]:
# 앞으로 해야할 것
# (1) upsample / downsample 잘해서 불균형 최대한 해결하기
	# - (local에서 나온 f1 score랑 실제 sumbit해서 나온 f1 score가 비슷해지면 잘 해결되는 느낌인 것 같음)
# (2) 무슨 scaler 쓰면 좋을지 연속형 변수같은 경우 hist 그리면서 파악하면 좋을 듯 (전처리하면서는 min max가 일단 나은 듯?)
# (3) 범주형 변수는 어떻게 encoding 어떻게 하면 좋을지

In [7]:
############################# IMPORT LIBRARY  #################################
import os
import random
import re
import tqdm
from tqdm.notebook import tqdm
from collections import Counter
from datetime import datetime
import argparse
import pickle
import logging
import numpy as np
import pandas as pd

# https://contrib.scikit-learn.org/category_encoders/index.html
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, QuantileTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

from sklearn.feature_selection import RFE, SelectFromModel, SelectKBest, f_classif, chi2
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score, precision_score, recall_score

from ctgan import CTGAN
from sklearn.neural_network import MLPRegressor

import optuna
from optuna.samplers import TPESampler, NSGAIISampler, NSGAIIISampler
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 200

c:\Users\1004c\miniconda3\envs\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
#######################   CONFIG  #######################
parser = argparse.ArgumentParser(description='Anomaly Detection')

parser.add_argument('--data_path', type=str, default='c:\\Users\\1004c\\Desktop\\LG - Aimers\\data\\! Final')
parser.add_argument('--seed',type=int, default=110)

parser.add_argument('--model', type=str, default='cat')

parser.add_argument('-en', '--encoder', type=str, default='js')
parser.add_argument('-s', '--scaler', type=str, default='qt')

downsample_options = {1:"nearmiss", 2:"cluster", 3:"allknn", 4:"oneside", 5:"tomek"}
parser.add_argument('-ds', '--downsampling', type=int, default=4) # OSS

upsample_options = {1: "random", 2:"smote", 3:"adasyn", 4:"smotenc", 5:"smoten", 6:"borderline", 7:"kmeans", 8:"svm"}
	# upsample의 option으로 CTGAN, VAE를 사용하는 경우
parser.add_argument('-us', '--upsampling', type=int, default=2) # SMOTEE - NC


####### 추가 augmentation으로 CTGAN을 사용하는 경우 #############
augmentation_options = {1: "ctgan", 2: "vae"}
parser.add_argument('-ag', '--augmentation', type=int, default=1)



parser.add_argument('--fs_mode', type=bool, default=True, help='feature selection T:auto F:manual')
parser.add_argument('--estimator', type=str, default='rfc', help="using for feature selection")
parser.add_argument('--selector', type=str, default='rfe', help='auto feature selector') #rfe 35개로 수정

parser.add_argument('--k', type=int, default=10, help='k fold split') # feature selection에서 overfitting이 발생하면 k-fold 사용하는게 유리
parser.add_argument('--check_all', type=bool, default=True) # 모든 조합을 확인할지 여부 설정
parser.add_argument('--tune_mode', type=bool, default=True, help='optuna tuning') # hyperparameter tuning 여부를 설정

config = parser.parse_args([])

exp_config = f"{config.encoder}_{config.scaler}_{downsample_options[config.downsampling]}_{upsample_options[config.upsampling]}"
	# 각각 인코더, 스케일러, 다운샘플링, 업샘플링 방법을 나타내는 값

random.seed(config.seed)
np.random.seed(config.seed)

### Data

In [29]:
#######################   LOAD DATA  #######################
df_tr = pd.read_csv(os.path.join(config.data_path, "train_v2.csv"))
df_te = pd.read_csv(os.path.join(config.data_path, "test_v2.csv"))
df_list = [df_tr, df_te]

# Workorder (test에 있는데, train에는 없는 경우가 있어 그냥 제외) => v2에서는 workorder 제거
# 대신 Workorder Categeory 사용
# for df in df_list:
#    df.drop(["Workorder"], axis=1, inplace=True)

- categorical, numerical 구분

In [30]:
############################  FEATURE HANDLING  ###########################
## CATEGORICAL FEATURES
cat_features = ["Equipment_Dam",
                "Equipment_Fill1",
                "Equipment_Fill2",
                "Model.Suffix",
                "Workorder Category",
                "Chamber Temp. Judge Value_AutoClave"]

## BINNING FEATURES
bins_features = df_tr.columns[df_tr.columns.str.contains(r".*Bins.*")].tolist()
# Bins 열 만드는 데 사용된 열
from_bins_features = [re.sub(r'\s*Bins\s*', '', f).strip() for f in bins_features]

cat_features.extend(bins_features)

for df in df_list:
    df[cat_features] = df[cat_features].astype("category")

## NUMERICAL FEATURES
num_features = df_tr.select_dtypes(exclude=["category"]).columns.to_list()
num_features.remove("target")

## ALL FEATURES
all_features = num_features + cat_features

## TARGET ENCODING
df_tr["target"] = df_tr["target"].map({"Normal": 0, "AbNormal": 1})

## DATA SPLITTING 
X_tr, y_tr = df_tr.drop("target", axis=1), df_tr["target"]
X_te = df_te.drop("Set ID", axis=1)

- categorical 변수 encoding

In [28]:
# (1) le(label encoder): 범주형 데이터를 순서가 있는 정수로 변환
# (2) js(James-Stein encoding): 각 범주가 목표 변수와 관련된 평균 값을 이용하여 인코딩 (범주가 적을 때 효과적)
# (3) woe(weight of evidence encoding): 각 범주가 목표 변수에 대해 가지고 있는 증거의 가중치를 계산 (이진 분류에 효과적)
# (4) ohe(one hot encoding): 모든 범주를 고유한 이진 벡터로 표현하기 때문에 범주 간의 순서나 크기를 부여하지 X

In [32]:
#############################  FEATURE ENCODING/SCALING ###########################
## ENCODING
if config.encoder == "le":
    le = LabelEncoder()
    for cat_feature in cat_features:
        X_tr[cat_feature] = le.fit_transform(X_tr[cat_feature])
        X_te[cat_feature] = le.transform(X_te[cat_feature])
        
elif config.encoder == "js":
    js = ce.JamesSteinEncoder(cols=cat_features)
    
    X_tr = js.fit_transform(X_tr, y_tr)
    X_te = js.transform(X_te)
    
elif config.encoder == "woe":
    woe = ce.WOEEncoder(cols=cat_features)
    
    X_tr = woe.fit_transform(X_tr, y_tr)
    X_te = woe.transform(X_te)
    
elif config.encoder == "ohe":
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
    
    X_tr[cat_features] = ohe.fit_transform(X_tr[cat_features])
    X_te[cat_features] = ohe.transform(X_te[cat_features])

- Scaling

In [30]:
# scaling: 서로 다른 변수의 값 범위를 일정한 수준으로 맞추는 작업
# scaling은 수치 데이터에만 적용 가능하기 때문에 일반적으로 encoding -> scaling 순으로 진행

# (1) mms(Min-Max Scaler): 데이터를 최소값 0과 최대값 1 사이로 변환
# (2) ss(Standard Scaler): 데이터를 평균 0, 표준편차 1로 변환
# (3) qt(quantile transformer): 데이터를 균등하게 분포된 값으로 변환 (이상치에 대한 민감도가 낮음. 정규분포에 가깝게 만들 때 유용)
# (4) pt(power transformer): 데이터의 분포를 정규분포에 가깝게 만들어주는데 효과적

In [33]:
## SCALING
if config.scaler == "mms":
    mms = MinMaxScaler()
    X_tr[num_features] = mms.fit_transform(X_tr[num_features])
    X_te[num_features] = mms.transform(X_te[num_features])
    
elif config.scaler == "ss":
    ss = StandardScaler()
    X_tr[num_features] = ss.fit_transform(X_tr[num_features])
    X_te[num_features] = ss.transform(X_te[num_features])
    
elif config.scaler == "qt":
    qt = QuantileTransformer(random_state=config.seed, output_distribution='normal', n_quantiles=min(100, len(X_tr) // 5)) # n_quantiles = 1000
    
    X_tr[num_features] = qt.fit_transform(X_tr[num_features])
    X_te[num_features] = qt.transform(X_te[num_features])

elif config.scaler == "pt":
    pts = PowerTransformer(method='yeo-johnson')
    
    X_tr[num_features] = pts.fit_transform(X_tr[num_features])
    X_te[num_features] = pts.transform(X_te[num_features])

### Data Upsampling/Downsampling

In [32]:
# (1) downsample: sample_ratio에 따라 'abnormal' 크기에 비례해 'normal' 클래스의 데이터 샘플 수를 조정
	# - nearmiss: 가장 멀리 떨어진 데이터를 제거해 다운샘플링
	# - cluster: 데이터의 중심을 기반으로 클러스터링하여 다운샘플링
	# - allknn: KNN 기반으로 중복 데이터를 제거해 다운샘플링
	# - oneside: 불균형 클래스의 데이터를 제거해 샘플링
	# - tomek: 두 클래스 간의 중첩된 데이터를 제거해 다운샘플링
# (2) upsample
	# - random: 랜덤하게 소수 클래스의 데이터를 복제하여 업샘플링
	# - smote: 소수 클래스 데이터의 근접 이웃을 사용해 새로운 샘플을 생성하여 업샘플링
	# - adassyn: 소수 클래스의 데이터를 생성할 때, 그 필요성에 따라 다르게 생성
	# - smotenc: smote와 비슷 + 범주형 변수도 처리 가능
	# - borderline: 경계 근처의 데이터에 집중해 smote 수행
	# - kmeans: KMeans clustering 기반으로 업샘플링
	# - svm: svm 기반 smote 방법

In [34]:
from collections import Counter
from sklearn.utils import resample
from imblearn.under_sampling import (NearMiss,
                                     ClusterCentroids,
                                     AllKNN,
                                     OneSidedSelection,
                                     TomekLinks)
from imblearn.over_sampling import (RandomOverSampler,
                                    SMOTE,
                                    ADASYN,
                                    SMOTENC,
                                    SMOTEN,
                                    BorderlineSMOTE,
                                    KMeansSMOTE,
                                    SVMSMOTE)
from imblearn.combine import SMOTEENN
# https://imbalanced-learn.org/stable/references/index.html


    # (1) random_downsample: 다수 클래스의 데이터를 random하게 선택해 줄이는 방식
    # (2) downsample: 여러 가지 알고리즘을 이용해 다수 클래스의 데이터를 선택적으로 줄이는 것


def random_downsample(df, random_seed, sample_ratio=1.0):
    df_normal = df[df["target"] == 0] 
    df_abnormal = df[df["target"] == 1]
    
    downsampled = resample(
        df_normal,
        replace=False,
        n_samples=int(len(df_abnormal) * sample_ratio),
        random_state=random_seed
    )
    
    downsampled_df = pd.concat([df_abnormal, downsampled])
    
    return downsampled_df

def downsample(X, y, method, random_seed):
    # NearMiss
    if method == "nearmiss":
        # sampling_strategy="auto"
        nm = NearMiss(sampling_strategy=0.4)
        X_downsampled, y_downsampled = nm.fit_resample(X, y)
    # ClusterCentroids
    elif method == "cluster":
        cc = ClusterCentroids(random_state=random_seed)
        X_downsampled, y_downsampled = cc.fit_resample(X, y)
    # AllKNN
    elif method == "allknn":
        allknn = AllKNN()
        X_downsampled, y_downsampled = allknn.fit_resample(X, y)
    # OneSidedSelection
    elif method == "oneside":
        oss = OneSidedSelection(random_state=random_seed)
        X_downsampled, y_downsampled = oss.fit_resample(X, y)
    # Tomeklinks
    elif method == "tomek":
        tl = TomekLinks()
        X_downsampled, y_downsampled = tl.fit_resample(X, y)
    
    X_downsampled_df= pd.DataFrame(X_downsampled, columns=X.columns)
    y_downsampled_df = pd.Series(y_downsampled, name="target") 
    downsampled_df = pd.concat([X_downsampled_df, y_downsampled_df], axis=1)
    
    print('DOWN SAMPLING')
    print('=============')
    print('Original dataset shape %s' % Counter(y))
    print('Resampled dataset shape %s' % Counter(y_downsampled), end='\n')
    
    return downsampled_df


def upsample(X, y, cat_idx, method, random_seed):
    
    if method == "random":
        ros = RandomOverSampler(random_state=random_seed)
        X_upsampled, y_upsampled = ros.fit_resample(X, y)
        
    # SMOTE
    elif method == "smote":
        smote = SMOTE(random_state=random_seed)
        X_upsampled, y_upsampled = smote.fit_resample(X, y)
        
    # ADASYN
    elif method == "adasyn":
        adasyn = ADASYN(random_state=random_seed)
        X_upsampled, y_upsampled = adasyn.fit_resample(X, y)
        
    # SMOTE-NC
    elif method == "smotenc":
        smotenc = SMOTENC(random_state=random_seed, sampling_strategy="auto", categorical_features=cat_idx)
        X_upsampled, y_upsampled = smotenc.fit_resample(X, y)
        
    elif method == "smoten":
        smoten = SMOTEN(random_state=random_seed, sampling_strategy="auto", k_neighbors=5)
        X_upsampled, y_upsampled = smoten.fit_resample(X, y)
        
    elif method == "borderline":
        borderline_smote = BorderlineSMOTE(random_state=random_seed)
        X_upsampled, y_upsampled = borderline_smote.fit_resample(X, y)
        
    elif method == "kmeans":
        kmeans_smote = KMeansSMOTE(random_state=random_seed, sampling_strategy="auto", k_neighbors=5)
        X_upsampled, y_upsampled = kmeans_smote.fit_resample(X, y)
        
    elif method == "svm":
        svm_smote = SVMSMOTE(random_state=42)
        X_upsampled, y_upsampled = svm_smote.fit_resample(X, y)
        
    X_upsampled_df= pd.DataFrame(X_upsampled, columns=X.columns)
    y_upsampled_df = pd.Series(y_upsampled, name="target") 
    upsampled_df = pd.concat([X_upsampled_df, y_upsampled_df], axis=1)
    
    print('UP SAMPLNG')
    print('==========')
    print('Original dataset shape %s' % Counter(y))
    print('Resampled dataset shape %s' % Counter(y_upsampled), end='\n')
    
    return upsampled_df

In [35]:
#################################  DOWN SAMPLING  ###############################
downsampled_df_tr = downsample(X_tr, y_tr, method=downsample_options[config.downsampling], random_seed=config.seed)
	#(1) config에 저장되어있지 않은 다른 method를 사용하고 싶으면 method에 다른 알고리즘 이름 넣기

#################################  UP SAMPLING  ###############################
cat_idx = [downsampled_df_tr.columns.get_loc(col) for col in cat_features]
cat_idx = [X_tr.columns.get_loc(col) for col in cat_features]
X_tr = downsampled_df_tr.drop("target", axis=1)
y_tr = downsampled_df_tr["target"]

upsampled_df_tr = upsample(X_tr, y_tr, cat_idx=cat_idx, method=upsample_options[config.upsampling], random_seed=config.seed)

DOWN SAMPLING
Original dataset shape Counter({0: 38155, 1: 2350})
Resampled dataset shape Counter({0: 37103, 1: 2350})
UP SAMPLNG
Original dataset shape Counter({0: 37103, 1: 2350})
Resampled dataset shape Counter({0: 37103, 1: 37103})


In [36]:
## RESAMPLED DATA
# X_tr = downsampled_df_tr.drop("target", axis=1)
# y_tr = downsampled_df_tr["target"]

X_tr = upsampled_df_tr.drop("target", axis=1)
y_tr = upsampled_df_tr["target"]

### Data Augmentation

In [39]:
# from ctgan import CTGAN
# from sklearn.neural_network import MLPRegressor
# from sdv.tabular import TVAE

def augmentation(X, y, method, random_seed):
    if method == "ctgan":
        X = pd.DataFrame(X)
        # Assuming cat_idx is the list of indices of categorical features
        discrete_columns = X.columns[cat_idx]  # Get the names of categorical columns

        # Fit the CTGAN model on the original data
        ctgan = CTGAN(epochs=100)
        ctgan.fit(X, discrete_columns=discrete_columns)

        # Generate synthetic samples
        synthetic_samples = ctgan.sample(X.shape[0])

        # Combine original and synthetic samples
        X_augmented = np.vstack([X, synthetic_samples])
        y_augmented = np.concatenate([y, np.ones(synthetic_samples.shape[0])])

    elif method == "vae":
        # Define a simple autoencoder architecture
        vae = MLPRegressor(hidden_layer_sizes=(X.shape[1] * 2, X.shape[1]), random_state=random_seed)
        # Fit the VAE on the minority class
        X_minority = X[y == 1]
        vae.fit(X_minority, X_minority)

        # Generate synthetic samples by predicting with the VAE
        synthetic_samples = vae.predict(X_minority)

        # Combine original and synthetic samples
        X_augmented = np.vstack([X, synthetic_samples])
        y_augmented = np.concatenate([y, np.ones(synthetic_samples.shape[0])])

    # elif method == "tvae":
    #     # Assuming X is a pandas DataFrame with both numerical and categorical columns
    #     tvae = TVAE(epochs=100, random_state=random_seed)

    #     # Fit the TVAE model on the original data
    #     tvae.fit(X)

    #     # Generate synthetic samples
    #     synthetic_samples = tvae.sample(X.shape[0])

    #     # Combine original and synthetic samples
    #     X_augmented = np.vstack([X, synthetic_samples])
    #     y_augmented = np.concatenate([y, np.ones(synthetic_samples.shape[0])])

    # return X_augmented, y_augmented


In [40]:
augmented_df_tr = augmentation(X_tr, y_tr, method=augmentation_options[config.augmentation], random_seed=config.seed)

## AUGMENTED DATA
X_tr = augmented_df_tr.drop("target", axis=1)
y_tr = upsampled_df_tr["target"]

MemoryError: Unable to allocate 2.27 GiB for an array with shape (74206, 8222) and data type int32

### Train - Val split

### Model

In [ ]:
# (1) ML, decision tree 기반 모델이 classification task에 유리

In [32]:
################ MODEL ############### 
classifiers = {
    "cat": CatBoostClassifier(random_state=config.seed, auto_class_weights="Balanced"),
    "lgbm": LGBMClassifier(random_state=config.seed,),
    "xgb": XGBClassifier(random_state=config.seed, eval_metric='auc', objective="binary:logistic"),
    "ada": AdaBoostClassifier(random_state=config.seed),
    "rfc": RandomForestClassifier(random_state=config.seed, class_weight='balanced'),
    "lr": LogisticRegression(random_state=config.seed),
    "extra": ExtraTreesClassifier(random_state=config.seed)
}
    #(1) 각 모델이 딕셔너리의 특정 키에 매핑되어 있으며, 이를 통해 나중에 원하는 모델을 쉽게 접근하고 사용할 수 있음

### Feature Selection

In [ ]:
# (1) RFE(Recursive Feature Elimination): 가장 중요하지 않은 특징을 반복적으로 제거해 최종적으로 지정한 수의 특징만 남기는 방법
# (2) selectfrom model: 모델의 중요도(feature importance)를 기반으로 특징을 선택
# (3) SelectKBest: 주어진 점수 함수(f_classif, ANOVA F-value)로 평가해 상위 k=20개의 특징을 선택

In [28]:
###############################  FEATURE SELECTION  ############################
if config.fs_mode:
    estimator = classifiers[config.estimator]
    estimator.fit(X_tr, y_tr)
    
    selectors = {
        'rfe': RFE(estimator=estimator, n_features_to_select=35),
        'sfm': SelectFromModel(estimator=estimator, threshold="mean"),
        'kbest': SelectKBest(score_func=f_classif, k=10),
    }
    
    selector = selectors[config.selector]
    
    # Fit the selector on the training data
    selector.fit(X_tr, y_tr)
    
    # Get the support mask of selected features
    support_mask = selector.get_support()
    
    # Retain selected features from the original DataFrame and preserve feature names
    X_tr_selec = X_tr.loc[:, support_mask]
    X_te_selec = X_te.loc[:, support_mask]
    
else:
    # 기존 열 대신 Bins 열 사용
    selected_features = [feature for feature in all_features if feature not in from_bins_features]
    
    X_tr_selec = X_tr[selected_features]
    X_te_selec = X_te[selected_features]
    
print("FEATRUE SELECTION")
print("Before ", X_tr.shape)
print("After ", X_tr_selec.shape, end='\n')

FEATRUE SELECTION
Before  (59432, 173)
After  (59432, 10)


In [29]:
# 어떤 feature selected 됐는지 확인
X_tr_selec.columns

Index(['Head Clean Position Z Collect Result_Dam',
       'Head Purge Position Z Collect Result_Dam',
       'Stage1 Circle1 Distance Speed Collect Result_Dam',
       'WorkMode Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result Bins_Dam',
       'Dispense Volume(Stage1) Collect Result Bins_Dam',
       'Dispense Volume(Stage3) Collect Result Bins_Dam',
       'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result Bins_Dam',
       'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result Bins_Dam',
       'Machine Tact time Collect Result Bins_Fill1'],
      dtype='object')

- 성능 비교

In [ ]:
###############################  EVALUATION  ############################
    #(1) train set에 대해 내부 검증 => overfitting이 되고 있는지에 대해 확인
    #(2) 모델 간의 성능을 평가하고 비교하는데 초점이 있음

#stk = StratifiedKFold(n_splits=10, random_state=config.seed, shuffle=True) #(1) 각 fold에서 클래스의 비융을 유지하면서 데이터셋을 n_splits 개의 subset으로 나누는 것
#rstk = RepeatedStratifiedKFold(n_splits=10, random_state=config.seed) #(2) StratifiedKFold를 여러 번 반복하여 더 안정적인 성능 평가를 제공하는 것

#if config.check_all:
#    classifiers_lst = list(classifiers.values())
#    score_df = pd.DataFrame(columns=classifiers.keys())
    
#    for clf_name, clf in classifiers.items():
#        scores = cross_val_score(clf, X_tr_selec, y_tr, scoring="f1", cv=stk)
#        print(scores)
#        score_df.loc[0, clf_name] = scores.mean()
#        print(score_df)
    
    
#else:
#    metrics = ['accuracy', 'precision', 'recall', 'f1', 'f1_weighted', 'roc_auc']
#    score_df = pd.DataFrame(columns=metrics)
    
#   for metric in metrics:
#       scores = cross_val_score(classifiers["cat"], X_tr_selec, y_tr, scoring=metric, cv=stk)
#       score_df[metric] = scores.mean()
    
#print("MODEL CHECK")
#print(score_df, end='\n')

- Parameter optimization

In [ ]:
#(1) optuna의 'trial' 객체를 사용하여 하이퍼파라미터를 제안하고 이를 바탕으로 모델을 학습
#(2) 위 코드와 다르게 여러 하이퍼파라미터를 탐색하며 모델을 최적화

def catboost_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
        'od_type': trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        'od_wait': trial.suggest_int("od_wait", 10, 50),
    }

    cat_clf = CatBoostClassifier(**params, random_state=config.seed, auto_class_weights="Balanced",) # eval_metric="TotalF1"
    
    stk = StratifiedKFold(n_splits=config.k, random_state=config.seed, shuffle=True)
    f1_scores = np.empty(config.k)
    
    for idx, (tr_idx, val_idx) in enumerate(stk.split(X_tr_selec, y_tr)):
        X_tr_fold, X_val_fold = X_tr.iloc[tr_idx], X_tr.iloc[val_idx]
        y_tr_fold, y_val_fold = y_tr.iloc[tr_idx], y_tr.iloc[val_idx]
        
        cat_clf.fit(X_tr_fold, y_tr_fold, eval_set=[(X_val_fold, y_val_fold)], early_stopping_rounds=50, verbose=False)
        y_pred_fold = cat_clf.predict(X_val_fold)
        f1_scores[idx] = f1_score(y_val_fold, y_pred_fold)

    return np.mean(f1_scores) 

In [ ]:
def lgbm_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
    }
    
    lgbm_clf = LGBMClassifier(**params, random_state=config.seed)
    
    stk = StratifiedKFold(n_splits=config.k, random_state=config.seed, shuffle=True)
    f1_scores = np.empty(config.k)
    
    for idx, (tr_idx, val_idx) in enumerate(stk.split(X_tr_selec, y_tr)):
        X_tr_fold, X_val_fold = X_tr_selec.iloc[tr_idx], X_tr_selec.iloc[val_idx]
        y_tr_fold, y_val_fold = y_tr.iloc[tr_idx], y_tr.iloc[val_idx]
        
        lgbm_clf.fit(X_tr_fold, y_tr_fold)
        y_pred_fold = lgbm_clf.predict(X_val_fold)
        f1_scores[idx] = f1_score(y_val_fold, y_pred_fold)

    return np.mean(f1_scores)

In [ ]:
def xgboost_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),  # colsample_bylevel -> colsample_bytree
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),  # XGBoost의 중요한 파라미터 중 하나
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),  # 트리의 분할을 조정하는 파라미터
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),  # L1 정규화 항
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),  # L2 정규화 항
    }

    xgb_clf = XGBClassifier(**params, random_state=config.seed, use_label_encoder=False,
                            eval_metric='auc',
                            early_stopping_rounds=50, 
                            objective = "binary:logistic",)
    
    stk = StratifiedKFold(n_splits=config.k, random_state=config.seed, shuffle=True)
    f1_scores = np.empty(config.k)
    
    for idx, (tr_idx, val_idx) in enumerate(stk.split(X_tr_selec, y_tr)):
        X_tr_fold, X_val_fold = X_tr.iloc[tr_idx], X_tr.iloc[val_idx]
        y_tr_fold, y_val_fold = y_tr.iloc[tr_idx], y_tr.iloc[val_idx]
        
        xgb_clf.fit(X_tr_fold, y_tr_fold, eval_set=[(X_val_fold, y_val_fold)], early_stopping_rounds=50, verbose=False)
        y_pred_fold = xgb_clf.predict(X_val_fold)
        f1_scores[idx] = f1_score(y_val_fold, y_pred_fold)

    return np.mean(f1_scores)

In [ ]:
# (1) sampler: 다목적 최적화를 위한 알고리즘 => 여러 목적을 동시에 고려해 하이퍼파라미터를 탐색
# (2) pruner: 비효율적인 학습을 일찍 중단시켜 자원을 절약하는 역할

# => 위에서 정의한 optimization 방식에 따라 아래에서 선택해서 optimization 진행

- parameter optimization: 일단 실행 X

In [ ]:
# logger = logging.getLogger()
    
# logger.setLevel(logging.INFO)
# logger.addHandler(logging.FileHandler(f"./log/{config.model}_optuna.log", mode="w"))
    
# optuna.logging.enable_propagation() 
# optuna.logging.disable_default_handler()
        
# sampler = NSGAIISampler(seed=config.seed)
# pruner = optuna.pruners.HyperbandPruner()

# if config.tune_mode and config.model == "cat":
    
#    cat_study = optuna.create_study(study_name="cat", direction='maximize', sampler=sampler, pruner=pruner)
#    cat_study.optimize(catboost_objective, n_trials=15)
    
#    cat_best_params = cat_study.best_params
#    cat_best_score = cat_study.best_value
    
#    print("CatBoost Best Hyperparams: ", cat_best_params)
#    print("CatBoost Best F1 Score: ", cat_best_score, end='\n')
    
#    final_clf = CatBoostClassifier(**cat_best_params, random_state=config.seed, auto_class_weights="Balanced",)
    
#elif config.tune_mode and config.model == "lgbm":
    
#    lgbm_study = optuna.create_study(study_name="lgbm", direction='maximize', sampler=sampler, pruner=pruner)
#    lgbm_study.optimize(lgbm_objective, n_trials=15)
    
#    lgbm_best_params = lgbm_study.best_params
#    lgbm_best_score= lgbm_study.best_value
    
#    print("LGBM Best Hyperparams: ",lgbm_best_params)
#    print("LGBM Best F1 Score: ", lgbm_best_score, end='\n')
    
#    final_clf = LGBMClassifier(**lgbm_best_params, random_state=config.seed,)

#elif config.tune_mode and config.model == "xgb":
    
#    xgb_study = optuna.create_study(study_name="xgb", direction='maximize', sampler=sampler, pruner=pruner)
#    xgb_study.optimize(xgboost_objective, n_trials=15)
    
#    xgb_best_params = xgb_study.best_params
#    xgb_best_score = xgb_study.best_value
#    print("XGBoost Best Hyperparams: ", xgb_best_params)
#    print("XGBoost Best F1 Score: ", xgb_best_score)
    
#else:
#    final_clf = classifiers[config.model]

# with open(f"./log/{config.model}_optuna.log") as f:
#     assert f.readline().startswith("A new study created")
#     assert f.readline() == "Start optimization.\n"

In [ ]:
#cat_study.visualize()
#cat_study.trials_dataframe()

### Model Stacking

In [34]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

stacking_clf = StackingClassifier(
    estimators=[
        ('cat', classifiers['cat']),
        ('xgb', classifiers['xgb']),
        ('extra', classifiers['extra'])
    ],
    final_estimator=XGBClassifier(random_state=config.seed, eval_metric='auc', objective="binary:logistic")
)

In [48]:
stacking_clf.fit(X_tr_selec, y_tr)

Learning rate set to 0.058944
0:	learn: 0.6899415	total: 19.1ms	remaining: 19.1s
1:	learn: 0.6868880	total: 40.5ms	remaining: 20.2s
2:	learn: 0.6839307	total: 60ms	remaining: 19.9s
3:	learn: 0.6807041	total: 76.4ms	remaining: 19s
4:	learn: 0.6787115	total: 94.7ms	remaining: 18.8s
5:	learn: 0.6745682	total: 114ms	remaining: 18.8s
6:	learn: 0.6722322	total: 129ms	remaining: 18.3s
7:	learn: 0.6704799	total: 144ms	remaining: 17.8s
8:	learn: 0.6683487	total: 160ms	remaining: 17.6s
9:	learn: 0.6650154	total: 174ms	remaining: 17.3s
10:	learn: 0.6631133	total: 190ms	remaining: 17.1s
11:	learn: 0.6601243	total: 203ms	remaining: 16.7s
12:	learn: 0.6587513	total: 220ms	remaining: 16.7s
13:	learn: 0.6573231	total: 236ms	remaining: 16.6s
14:	learn: 0.6551525	total: 252ms	remaining: 16.6s
15:	learn: 0.6540271	total: 265ms	remaining: 16.3s
16:	learn: 0.6525261	total: 280ms	remaining: 16.2s
17:	learn: 0.6505595	total: 298ms	remaining: 16.2s
18:	learn: 0.6497825	total: 313ms	remaining: 16.1s
19:	learn:

StackingClassifier(estimators=[('cat',
                                <catboost.core.CatBoostClassifier object at 0x000001ED3C93AB60>),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='auc',
                                              feature_types=None, gamma=None,
                                              grow_p...
                                                 grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=110, ...))

### Submission

In [49]:
################################################################
#####################     SUBMISSION   #########################
################################################################

# final_clf.fit(X_tr_selec, y_tr, ) # use_best_model=True
# final_preds = final_clf.predict(X_te_selec)

final_preds = stacking_clf.predict(X_te_selec) # stacking model을 사용하는 경우

df_sub = pd.read_csv(os.path.join(config.data_path, "submission.csv"))
df_sub["target"] = final_preds
df_sub["target"] = df_sub["target"].map({0 : "Normal", 1 : "AbNormal"})

print('=============================')
print(df_sub["target"].value_counts())

target
Normal      13153
AbNormal     4208
Name: count, dtype: int64


In [50]:
curr_date = datetime.now().strftime("%m-%d_%H-%M-%S")

# pickle.dump(final_clf, open(f"{config.model}_{curr_date}.pkl", "wb"))
# final_clf = pickle.load(open(".pkl", "rb"))
df_sub.to_csv(os.path.join(config.data_path, f"submission_{curr_date}_{exp_config}.csv"), index=False)